In [10]:
import random 

import joblib

import sys
sys.path.append('../..')

#from utils import load_cinema_reviews

import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargamos variables de ejercicios anteriores y formateamos ligeramente alguna de ellas

In [3]:
X_train = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/X_train.txt")
X_train = X_train.drop(columns= "Unnamed: 0")

X_test = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/X_test.txt")
X_test = X_test.drop(columns= "Unnamed: 0")

y_train = np.loadtxt("/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/y_train.txt")
y_test = np.loadtxt("/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/y_test.txt")

### Regresión logistica

Aplicamos un modelo de regresión logistica y la incluimos en un bucle GridSearchCV para obtener el parámetro de regularización más apropiado

In [6]:
grid = {"C" : [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000] }

grid = GridSearchCV(LogisticRegression(tol=1e-4, max_iter = 1000, random_state=42), param_grid = grid , cv = 5, scoring= 'accuracy', 
                    n_jobs = -1, verbose = 2, refit=True, return_train_score = True)
grid.fit(np.asarray(X_train), y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 38.6min finished


best mean cross-validation score: 0.870
best parameters: {'C': 1}


In [20]:
joblib.dump(grid, '/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/logistic_grid.pkl')

['/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/logistic_grid.pkl']

### MLP

In [12]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras import layers, Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from keras.models import Model

Dividimos el dataset entre train y CV para poder hacer el tunning de la red neuronal

In [13]:
tf_X_train, tf_X_CV, tf_y_train, tf_y_CV = train_test_split(X_train, y_train, train_size=0.75, test_size=0.25, random_state=42, shuffle=True)

In [14]:
shape= tf_X_train.shape[1]

Creamos el modelo

In [15]:
def create_mlp(shape):
# define our MLP network
  initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
  model = Sequential()
  model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
  model.add(Dropout(0.25))
  model.add(Dense(8, activation="relu"))
  model.add(Dropout(0.25))
  model.add(Dense(4, activation="relu"))
  model.add(Dropout(0.25))
  model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress: 
  model.add(Dense(1, activation="sigmoid"))
    #Compile model 
  opt = keras.optimizers.Adam(learning_rate = 0.001)    
  model.compile(loss='binary_crossentropy', metrics ="accuracy", optimizer=opt)
# return our model
  return model

Se han probado diversas combinaciones a la hora de hacer el modelo. La que se muestra justo sobre estas lineas, es la final. Durante el proceso se ha variado el numero de epocas y el learning rate hasta dar con los valores más óptimos. Tambien se ha añadido varias capas "Dropout" para compensar el overfitting presente ( se observaba una diferencia mayor al 10% entre el parametro "accuracy" de train y la de "validation")

In [17]:
mlp = create_mlp(shape)
history = mlp.fit(x=np.asarray(tf_X_train), y=tf_y_train,
    validation_data=(np.asarray(tf_X_CV), tf_y_CV),
    epochs=10,
    workers = -1, use_multiprocessing= True, verbose = 2)

Epoch 1/10
981/981 - 4s - loss: 0.6238 - accuracy: 0.6576 - val_loss: 0.4714 - val_accuracy: 0.8282
Epoch 2/10
981/981 - 2s - loss: 0.4888 - accuracy: 0.7915 - val_loss: 0.3966 - val_accuracy: 0.8462
Epoch 3/10
981/981 - 2s - loss: 0.4381 - accuracy: 0.8182 - val_loss: 0.3647 - val_accuracy: 0.8580
Epoch 4/10
981/981 - 2s - loss: 0.3927 - accuracy: 0.8438 - val_loss: 0.3653 - val_accuracy: 0.8505
Epoch 5/10
981/981 - 2s - loss: 0.3548 - accuracy: 0.8602 - val_loss: 0.3451 - val_accuracy: 0.8613
Epoch 6/10
981/981 - 2s - loss: 0.3210 - accuracy: 0.8751 - val_loss: 0.3415 - val_accuracy: 0.8630
Epoch 7/10
981/981 - 2s - loss: 0.3014 - accuracy: 0.8862 - val_loss: 0.3494 - val_accuracy: 0.8654
Epoch 8/10
981/981 - 2s - loss: 0.2832 - accuracy: 0.8934 - val_loss: 0.3510 - val_accuracy: 0.8629
Epoch 9/10
981/981 - 2s - loss: 0.2659 - accuracy: 0.9026 - val_loss: 0.3598 - val_accuracy: 0.8646
Epoch 10/10
981/981 - 2s - loss: 0.2450 - accuracy: 0.9112 - val_loss: 0.3680 - val_accuracy: 0.8601

Guardamos el modelo para analizarlo en el siguiente ejercicio

In [27]:
joblib.dump(history.history, '/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/history.pkl')
mlp.save('/content/drive/MyDrive/Colab_Notebooks/Proyecto_NLP/mlp.h5')

En el siguiente ejercicio analizaremos las métricas de cada modelo y decidiremos cual es el mejor de los dos para la clasificación binaria de reviews